In [48]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import gensim
import numpy as np
import pandas as pd
device = torch.device("cpu")

from torch.utils.data.dataset import Dataset

In [49]:
from konlpy.tag import Kkma
kkma = Kkma()

ko_model = gensim.models.Word2Vec.load('./ko/ko.bin')

In [171]:
from konlpy.tag import Komoran
komoran = Komoran()

In [231]:
class TargetDataSet(Dataset):
    def __init__(self,csv_file):
        self.Target = pd.read_csv(csv_file,sep = '\t',header = None,error_bad_lines=False)
    
    def __len__(self):
        return len(self.Target)
    def __getitem__(self, idx):
        data = []
        texts = self.Target[0][idx]
        sen_split = komoran.morphs(texts)
#         print(sen_split)
        w2v_list = []
        for i in range(20):
            if len(sen_split)>i:
                try:
                    w2v = ko_model.wv[sen_split[i]]
                    w2v_list.append(w2v)
                except:
                    w2v_list.append(np.zeros(200))
            else:
                w2v_list.append(np.zeros(200))
            
        w2v_list = np.array(w2v_list)
        data.append(w2v_list)
        data = np.array(data)
        data = data.reshape(20,200)
        data = torch.from_numpy(data.astype('float32'))
        label = torch.from_numpy(np.array(self.Target[1][idx]))
        
        return data,label

In [232]:
batch_size = 4

TARGET_DATASET_TRAIN = TargetDataSet(csv_file = './data/traindata.txt')
# TARGET_DATASET_VAL   = TargetDataSet(csv_file = 'traindata.txt')

TARGET_DATASET_TRAIN_LODER = torch.utils.data.DataLoader(TARGET_DATASET_TRAIN, batch_size = batch_size, shuffle = True, num_workers = 0)
# TARGET_DATASET_VAL_LODER = torch.utils.data.DataLoader(TARGET_DATASET_TRAIN, batch_size = batch_sizem shuffle = True, num_workers = 0)

b'Skipping line 79: expected 2 fields, saw 3\nSkipping line 175: expected 2 fields, saw 3\n'


In [233]:
TARGET_DATASET_TRAIN[0]

(tensor([[ 0.3369, -0.1450,  0.3567,  ..., -0.2557,  0.6565,  0.0740],
         [-1.2767,  1.1824, -1.2436,  ...,  1.3085,  0.0848, -1.5699],
         [ 1.3817,  1.0780,  1.9430,  ...,  2.1113, -2.5899,  1.2127],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]),
 tensor(1))

In [234]:
TARGET_DATASET_TRAIN[0][0].shape

torch.Size([20, 200])

In [235]:
dataiter = iter(TARGET_DATASET_TRAIN_LODER)
vector, labels = dataiter.next()
# print(vector,labels)
vector.shape

torch.Size([4, 20, 200])

In [236]:
input_size  = 200
hidden_size = 1
batch_size  = 4
seq_len     = 20

num_layer   = 1
num_classes = 2


learning_rate = 0.001
#==================Test용 input hidden입니다============================
inputs  = Variable(torch.randn(batch_size,seq_len,input_size))
hidden = Variable(torch.zeros(1,batch_size,hidden_size))

In [237]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        # Set initial hidden and cell states 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return F.softmax(out,dim=-1)

In [238]:
check = False
if(check == True):
    model = RNN(input_size, hidden_size, num_layer, num_classes)
    model.load_state_dict(torch.load("T2W"))
else:
    model = RNN(input_size, hidden_size, num_layer, num_classes)

In [239]:

model.train()
criterion = nn.CrossEntropyLoss()
# criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(100):
    TRN_LOSS = 0
    for i,data in enumerate(TARGET_DATASET_TRAIN_LODER,0):
        inputs, labels = data
        inputs, labels = Variable(inputs),Variable(labels)
        
        optimizer.zero_grad()
        output = model.forward(inputs)
        loss   = criterion(output,labels)
        loss.backward()
        optimizer.step()
        
        TRN_LOSS +=loss.item()

In [240]:
torch.save(model.state_dict(), "T2W")

# TEST

In [259]:
texts = '엄마가 다치셨는데 걱정하지 말고 아줌마랑 가자'
# sen_split = kkma.morphs(texts)
sen_split = komoran.morphs(texts)
sen_split

['엄마', '가', '다치', '시', '었', '는데', '걱정', '하', '지', '말', '고', '아줌마', '랑', '가자']

In [260]:
#=========나중에 실시간으로 들어오는거 처리할때 무조건 필요하니까 절대 지우면 안된다!!!=========


w2v_list = []
for i in range(20):
    if len(sen_split)>i:
        print(len(sen_split))
        try:
            w2v = ko_model.wv[sen_split[i]]
            w2v_list.append(w2v)
        except:
            print("asd")
            w2v_list.append(np.zeros(200))
    else:
#         print("asd")
        w2v_list.append(np.zeros(200))
    
w2v_list = np.array(w2v_list)
w2v_tensor    = torch.from_numpy(w2v_list.astype('float32'))
w2v_tensor_ch =torch.reshape(w2v_tensor,(1,w2v_tensor.size()[0],w2v_tensor.size()[1]))
w2v_tensor_ch.size()
w2v_tensor_ch

14
14
14
14
14
14
14
14
14
14
14
14
14
14


tensor([[[ 0.1929, -0.6049,  1.5089,  ...,  0.0597,  0.9609, -1.1660],
         [-2.4270, -1.5736,  0.8362,  ...,  0.4343,  1.4763, -0.1952],
         [ 0.1413, -0.3792, -0.1403,  ..., -0.7748,  0.6649, -0.6575],
         ...,
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]])

In [261]:
AA = model(w2v_tensor_ch)
AA

tensor([[0.0221, 0.9779]], grad_fn=<SoftmaxBackward>)

# 정확도 테스트

In [16]:
#test the model
model.eval()
train_correct = 0
train_total = 0

train_prediction_list = []
train_label_list = []


with torch.no_grad():
    for i,data in enumerate(TARGET_DATASET_TRAIN_LODER,0):
        train_data, train_label = data
        print()
        train_output = model.forward(train_data)
        _, train_prediction_index = torch.max(train_output, 1)
        
        train_prediction_list.append(train_prediction_index)
        train_label_list.append(train_label)
        
        train_total += train_label.size(0)
        train_correct += (train_prediction_index == train_label).sum().float()
    

    print("Train_Accuracy of the model: {}".format(train_correct/train_total))







Train_Accuracy of the model: 1.0


In [110]:
train_total

23

# 이 밑에 지우면 너가 죽는다

In [ ]:
class TargetDataSet(Dataset):
    def __init__(self,csv_file):
        self.Target = pd.read_csv(csv_file,sep = '\t',header = None,error_bad_lines=False)
    
    def __len__(self):
        return len(self.Target)
    def __getitem__(self, idx):
        data = []
        for sen_split in self.Target[0]:
            w2v_list = []
            for i in range(30):
#                 print(i)
                if len(sen_split)>i:
                    try:
                        w2v = ko_model.wv[sen_split[i]]
                        w2v_list.append(w2v)
                    except:
                        w2v_list.append(np.zeros(200))
                else:
                    w2v_list.append(np.zeros(200))
            
            w2v_list = np.array(w2v_list)
            data.append(w2v_list)
            
        data = np.array(data)
        data = torch.from_numpy(data.astype('float32'))
        
        label = torch.from_numpy(np.array(self.Target[1]))
        
        return data,label